In [1]:
import numpy as np, pandas as pd
import os,glob, re
%matplotlib inline

dfs = {re.search('([a-zA-Z_]*)\.csv', fn).group(1):pd.read_csv(fn) for fn in glob.glob(os.getcwd()+'//Raw//*.csv')}
print('data frames read:{}'.format(list(dfs.keys())))

print('local variables with the same names are created.')
for k, v in dfs.items(): locals()[k] = v

data frames read:['air_reserve', 'air_store_info', 'air_visit_data', 'date_info', 'hpg_reserve', 'hpg_store_info', 'sample_submission', 'store_id_relation']
local variables with the same names are created.


In [2]:
air_store_info['city'] = [x[0].split(' ')[0] for x in air_store_info['air_area_name'].str.split('-')]
air_store_info['loc'] = [x[1].split(' ')[1] for x in air_store_info['air_area_name'].str.split('-')]

In [3]:
air_visit_data = air_visit_data.merge(air_store_info,on = 'air_store_id',how = 'left')

In [4]:
# air_visit_data['dow'] = pd.to_datetime(air_visit_data['visit_date']).dt.dayofweek
# air_visit_data['woy'] = pd.to_datetime(air_visit_data['visit_date']).dt.weekofyear - 17
# air_visit_data['yr'] = pd.to_datetime(air_visit_data['visit_date']).dt.year
# air_visit_data.tail()

In [5]:
# air_visit_data.groupby(['yr','woy','city'])['air_store_id'].nunique().unstack().plot(figsize = (20,5))

In [6]:
# air_visit_data.groupby(['yr','woy','city'])['visitors'].mean().unstack().plot(figsize = (20,5))

In [7]:
print("Add features to date_info")
print()
print("1. Remove Saturday and Sunday as flagged holidays")
weekdayholidays = date_info.apply(lambda x: x.day_of_week in ['Saturday','Sunday'] and x.holiday_flg == 1,axis=1)
date_info.loc[weekdayholidays,'holiday_flg'] = 0
print("2. Add day before the holiday feature")
date_info['lagged'] = date_info['holiday_flg'].shift(-1)
day_b4_holiday = (date_info['holiday_flg'] == 0) & (date_info['lagged'] == 1.0) #& (date_info['day_of_week'] not ['Sunday','Saturday'])
date_info['dayb4holiday'] = 0
date_info.loc[day_b4_holiday,'dayb4holiday'] = 1
date_info.drop('lagged',axis = 1, inplace = True)
print("3. Add year, month, day of year features")
date_info['year'] = pd.to_datetime(date_info['calendar_date']).dt.year
date_info['month'] = pd.to_datetime(date_info['calendar_date']).dt.month
date_info['woy'] = pd.to_datetime(date_info['calendar_date']).dt.weekofyear
date_info['doy'] = pd.to_datetime(date_info['calendar_date']).dt.dayofyear
date_info.head()

Add features to date_info

1. Remove Saturday and Sunday as flagged holidays
2. Add day before the holiday feature
3. Add year, month, day of year features


,calendar_date,day_of_week,holiday_flg,dayb4holiday,year,month,woy,doy
0,2016-01-01,Friday,1,0,2016,1,53,1
1,2016-01-02,Saturday,0,0,2016,1,53,2
2,2016-01-03,Sunday,0,0,2016,1,53,3
3,2016-01-04,Monday,0,0,2016,1,1,4
4,2016-01-05,Tuesday,0,0,2016,1,1,5


In [8]:
goldenweek_2016start = '2016-04-29' 
goldenweek_2016end = '2016-05-05'
goldenweek_2017start = '2017-04-29' 
goldenweek_2017end = '2017-05-07'

date_info['golden'] = 0
date_info.loc[((date_info['calendar_date'] >= goldenweek_2016start) & (date_info['calendar_date'] <= goldenweek_2016end))
              | ((date_info['calendar_date'] >= goldenweek_2017start) & (date_info['calendar_date'] <= goldenweek_2017end))
              ,'golden'] = 1
date_info[date_info['golden'] == 1]

,calendar_date,day_of_week,holiday_flg,dayb4holiday,year,month,woy,doy,golden
119,2016-04-29,Friday,1,0,2016,4,17,120,1
120,2016-04-30,Saturday,0,0,2016,4,17,121,1
121,2016-05-01,Sunday,0,0,2016,5,17,122,1
122,2016-05-02,Monday,0,1,2016,5,18,123,1
123,2016-05-03,Tuesday,1,0,2016,5,18,124,1
124,2016-05-04,Wednesday,1,0,2016,5,18,125,1
125,2016-05-05,Thursday,1,0,2016,5,18,126,1
484,2017-04-29,Saturday,0,0,2017,4,17,119,1
485,2017-04-30,Sunday,0,0,2017,4,17,120,1
486,2017-05-01,Monday,0,0,2017,5,18,121,1


In [9]:
date_info.loc[date_info['golden'] == 1,'holiday_flg'] = 0
date_info.loc[date_info['golden'] == 1,'dayb4holiday'] = 0
date_info[date_info['golden'] == 1]

,calendar_date,day_of_week,holiday_flg,dayb4holiday,year,month,woy,doy,golden
119,2016-04-29,Friday,0,0,2016,4,17,120,1
120,2016-04-30,Saturday,0,0,2016,4,17,121,1
121,2016-05-01,Sunday,0,0,2016,5,17,122,1
122,2016-05-02,Monday,0,0,2016,5,18,123,1
123,2016-05-03,Tuesday,0,0,2016,5,18,124,1
124,2016-05-04,Wednesday,0,0,2016,5,18,125,1
125,2016-05-05,Thursday,0,0,2016,5,18,126,1
484,2017-04-29,Saturday,0,0,2017,4,17,119,1
485,2017-04-30,Sunday,0,0,2017,4,17,120,1
486,2017-05-01,Monday,0,0,2017,5,18,121,1


In [10]:
date_info['weights'] = ((date_info.index + 1)/ len(date_info))**7
date_info.tail()

,calendar_date,day_of_week,holiday_flg,dayb4holiday,year,month,woy,doy,golden,weights
512,2017-05-27,Saturday,0,0,2017,5,21,147,0,0.947082
513,2017-05-28,Sunday,0,0,2017,5,21,148,0,0.960081
514,2017-05-29,Monday,0,0,2017,5,22,149,0,0.973233
515,2017-05-30,Tuesday,0,0,2017,5,22,150,0,0.986539
516,2017-05-31,Wednesday,0,0,2017,5,22,151,0,1.000000


In [11]:
air_visit_data = air_visit_data.merge(date_info,left_on = 'visit_date',right_on='calendar_date',how= 'left')
air_visit_data.drop(['visit_date','calendar_date'],axis = 1,inplace = True)
air_visit_data.head()

,air_store_id,visitors,air_genre_name,air_area_name,latitude,longitude,city,loc,day_of_week,holiday_flg,dayb4holiday,year,month,woy,doy,golden,weights
0,air_ba937bf13d40fb24,25,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō,Minato,Wednesday,0,0,2016,1,2,13,0,6.355792e-12
1,air_ba937bf13d40fb24,32,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō,Minato,Thursday,0,0,2016,1,2,14,0,1.067732e-11
2,air_ba937bf13d40fb24,29,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō,Minato,Friday,0,0,2016,1,2,15,0,1.730633e-11
3,air_ba937bf13d40fb24,22,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō,Minato,Saturday,0,0,2016,1,2,16,0,2.718980e-11
4,air_ba937bf13d40fb24,6,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,Tōkyō,Minato,Monday,0,0,2016,1,3,18,0,6.201171e-11


In [20]:
air_visit_data.loc[air_visit_data['city'] == 'Fukuoka','loc'].unique()

array(['Fukuoka', 'Kitakyūshū', 'Kurume', 'Itoshima', 'Yame'], dtype=object)

In [19]:
air_visit_data[4445:]

,air_store_id,visitors,air_genre_name,air_area_name,latitude,longitude,city,loc,day_of_week,holiday_flg,dayb4holiday,year,month,woy,doy,golden,weights
4445,air_9438d67241c81314,32,Italian/French,Fukuoka-ken Fukuoka-shi Daimyō,33.589216,130.392813,Fukuoka,Fukuoka,Thursday,0,0,2016,11,47,329,0,0.042261
4446,air_9438d67241c81314,43,Italian/French,Fukuoka-ken Fukuoka-shi Daimyō,33.589216,130.392813,Fukuoka,Fukuoka,Friday,0,0,2016,11,47,330,0,0.043168
4447,air_9438d67241c81314,45,Italian/French,Fukuoka-ken Fukuoka-shi Daimyō,33.589216,130.392813,Fukuoka,Fukuoka,Saturday,0,0,2016,11,47,331,0,0.044092
4448,air_9438d67241c81314,23,Italian/French,Fukuoka-ken Fukuoka-shi Daimyō,33.589216,130.392813,Fukuoka,Fukuoka,Sunday,0,0,2016,11,47,332,0,0.045033
4449,air_9438d67241c81314,21,Italian/French,Fukuoka-ken Fukuoka-shi Daimyō,33.589216,130.392813,Fukuoka,Fukuoka,Monday,0,0,2016,11,48,333,0,0.045991
4450,air_9438d67241c81314,25,Italian/French,Fukuoka-ken Fukuoka-shi Daimyō,33.589216,130.392813,Fukuoka,Fukuoka,Tuesday,0,0,2016,11,48,334,0,0.046967
4451,air_9438d67241c81314,54,Italian/French,Fukuoka-ken Fukuoka-shi Daimyō,33.589216,130.392813,Fukuoka,Fukuoka,Wednesday,0,0,2016,11,48,335,0,0.047960
4452,air_9438d67241c81314,26,Italian/French,Fukuoka-ken Fukuoka-shi Daimyō,33.589216,130.392813,Fukuoka,Fukuoka,Thursday,0,0,2016,12,48,336,0,0.048971
4453,air_9438d67241c81314,43,Italian/French,Fukuoka-ken Fukuoka-shi Daimyō,33.589216,130.392813,Fukuoka,Fukuoka,Friday,0,0,2016,12,48,337,0,0.050001
4454,air_9438d67241c81314,23,Italian/French,Fukuoka-ken Fukuoka-shi Daimyō,33.589216,130.392813,Fukuoka,Fukuoka,Saturday,0,0,2016,12,48,338,0,0.051049
